In [1]:
# import
import numpy as np
from tqdm import tqdm
from pprint import pprint
import pickle

import torch
import torch.nn.functional as F
from datasets import load_from_disk
from transformers import AutoTokenizer, TrainingArguments

/opt/conda/envs/main/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 0 ~ 1.0 사이 값으로 scaling
def min_max_scaling(scores):
    min_score = torch.min(scores)
    max_score = torch.max(scores)
    scaled_scores = (scores - min_score) / (max_score - min_score)
    return scaled_scores

In [7]:
# top-k 확인
def get_relevant_doc(
        tokenizer,
        query,
        passage,
        ground_truth,
        p_name,
        q_name,
        k=1,
        args=None
    ): # 유사도 높은 context 및 ground-truth context와의 유사도 추출
    
    with open("../code/dense_encoder/" + p_name,  "rb") as f:
        p_encoder = pickle.load(f)
    with open("../code/dense_encoder/" + q_name, "rb") as f:
        q_encoder = pickle.load(f)

    with torch.no_grad():
        p_encoder.eval()
        q_encoder.eval()
        q_seps = tokenizer([query], padding="max_length", truncation=True, return_tensors="pt").to("cuda")
        q_emb = q_encoder(**q_seps).to("cpu") # (num_query, emb_dim)

        ground_truth = tokenizer(ground_truth, padding="max_length", truncation=True, return_tensors="pt").to("cuda")
        g_emb = p_encoder(**ground_truth).to("cpu")

        p_embs = []
        for p in passage:
            p = tokenizer(p, padding="max_length", truncation=True, return_tensors="pt").to("cuda")
            p_emb = p_encoder(**p).to("cpu").numpy()
            p_embs.append(p_emb)
    
    p_embs = torch.Tensor(p_embs).squeeze()  # (num_passage, emb_dim)
    dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1)) # question과의 유사도
    rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()

    # ground_truth와 passages의 유사도
    p_embs = F.normalize(p_embs, dim=-1)
    g_emb = F.normalize(g_emb, dim=-1)
    dot_prod_scores = torch.matmul(
        g_emb, torch.transpose(p_embs, 0, 1)).squeeze() # 각 문서간의 유사도
    # softmax = F.softmax(dot_prod_scores, dim=1).squeeze() # (num_passage,) → ground-truth 문서에 대한 전체 유사도
    dot_prod_scores = min_max_scaling(dot_prod_scores)

    return dot_prod_scores, rank[:k]

In [8]:
# document search
def search_doc(tokenizer, query, contexts, p_name, q_name): # query가 여럿일 경우 순서대로 검색
    context_score = []

    # for q in query
    for i, q in tqdm(enumerate(query), total=len(query), desc="Searching", unit="query"):
        score, indices = get_relevant_doc(
            tokenizer=tokenizer, query=q, passage=contexts, ground_truth=contexts[i], p_name=p_name, q_name=q_name, k=5)

        # print(f"[Search Query] {q}")
        # print(f'score : {scores}')

        arr = [[contexts[idx], score[idx]] for idx in indices] 
        context_score.append(arr) # [context, score]

    return context_score # (num_query, k, 2)  query마다 top-k개의 context와 score로 구성된 리스트 반환

        # top-k 문서 및 유사도 확인
        # for rank, idx in enumerate(indices):
        #     print(f"Top-{rank + 1}th Passage (Index {idx})")
        #     # pprint(retriever.passage['context'][idx])
        #     print(f"유사도 : {score[idx]}")
        #     pprint(contexts[idx])

In [ ]:
# wikipedia 불러오기
# with open("../data/wikipedia_documents.json", "r", encoding="utf-8") as f:
#     wiki = json.load(f)
# contexts = list(dict.fromkeys([
#             value["text"] for value in wiki.values()
#         ]))

model_checkpoint = "klue/bert-base"
p_name = "p_klue_bert_base_3.bin"
q_name = "q_klue_bert_base_3.bin"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

dataset = load_from_disk("../../../data/train_dataset")
val_dataset = dataset["validation"]["question"]
contexts = dataset["validation"]["context"]

# query와 비교하여 문서 찾기
query = val_dataset

context_score = search_doc(tokenizer, query, contexts, p_name, q_name)

/opt/conda/envs/main/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Searching:   0%|          | 1/240 [00:04<16:06,  4.04s/query]

[Search Query] 처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?


Searching:   1%|          | 2/240 [00:08<15:52,  4.00s/query]

[Search Query] 스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?


Searching:   1%|▏         | 3/240 [00:11<15:40,  3.97s/query]

...

[Search Query] 벽에 천녀를 그리기 전에 하는 밑작업은?


Searching:  23%|██▎       | 55/240 [03:43<12:29,  4.05s/query]

[Search Query] 합천에서 나루터 역할을 대신하고 있는 것은?


Searching:  23%|██▎       | 56/240 [03:47<12:27,  4.06s/query]

[Search Query] 독일의 취업자들이 주로 기술을 습득한 방법은?


In [ ]:
# (num_query, k, 2)  query마다 top-k개의 context와 score로 구성된 array
context_score = np.array(context_score)

# 유사도(score) 0.99 이상으로 정답 확인 or ground truth 직접 비교로 확인
# 이 부분은 시간이 없어서 업로드만 하고 PR
for con_sco in context_score:
    for context, score in con_sco:
        print(context)
        print(score.item())